# 시작

In [47]:
import os, glob, re
from pprint import pprint
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 데이터 준비

In [48]:
def read_lyrics():
    '''
    파일에서 가사들을 불러옴, LMS동일
    '''
    dir_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'
    result = []
    for file_path in glob.glob(dir_path):
        with open(file_path, 'r') as lyric_file:
            sentences = lyric_file.read().splitlines()
            result.extend(sentences)
    return result

def preprocess_sentence(sentence):
    '''
    문장들에서 특수 문자, 띄어쓰기 등을 보정한 후 <start>와 <end>를 문장 처음과 끝에 추가, LMS동일
    '''
    sentence = sentence.lower().strip()  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

def make_corpus(sentences):
    corpus = []
    print('원래 문장 TOP 10:')
    pprint(sentences[:10])
    print('\n')
    for sentence in sentences:
        if len(sentence) == 0: continue
        if sentence[-1] == ":": continue
        corpus.append(preprocess_sentence(sentence))
    print('특수 문자, 띄어쓰기 등을 보정한 문장 TOP 10:')
    pprint(corpus[:10])
    print('\n')
    return corpus

In [49]:
def tokenize(corpus, num_words=12000):
    '''
    문장을 단어 단위로 끊고, 토큰으로 생성함, LMS와 거의 같음
    
    corpus    : 문장
    num_words : 사용할 단어 수
    
    return    : 생성된 토큰 텐서, 토큰화에 사용된 맵
    '''
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=num_words, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = list(filter(lambda x: len(x) < 16, tensor)) # 토큰의 길이를 15개로 제한
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print('토큰화 + 패딩 후 문장 TOP 10:')
    pprint(tensor[:10])
    print('\n')
    print('토크나이저 TOP 10:')
    for index in tokenizer.index_word:
        print(tokenizer.index_word[index])
        if index > 10:
            break
    print('\n')
    return tensor, tokenizer

In [50]:
def make_single_dataset(x, y, batch_size):
    '''
    한 개의 데이터셋을 생성. tf.data.Dataset
    '''
    size = len(x)
    return tf.data.Dataset.from_tensor_slices((x, y)).shuffle(size).batch(batch_size, drop_remainder=True)
    
def make_dataset(source, target, batch_size=256):
    '''
    훈련, 검증 데이터셋을 생성. tf.data.Dataset
    
    source : 문장의 시작
    target : 문장의 다음
    
    return : 훈련 데이터셋, 검증 데이터셋
    '''
    train_x, test_x, train_y, test_y = train_test_split(source, target, test_size=0.2, shuffle=True)
    print('훈련, 검증 데이터 분리 후 shape: ', train_x.shape, train_y.shape)
    
    train_dataset = make_single_dataset(train_x, train_y, batch_size)
    val_dataset = make_single_dataset(test_x, test_y, batch_size)
    
    return train_dataset, val_dataset

# 모델

In [51]:
'''
학습에 사용할 모델
'''
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        
        # self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        # self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        
        self.rnn_1 = tf.keras.layers.GRU(hidden_size, dropout=0.3, return_sequences=True)
        self.rnn_2 = tf.keras.layers.GRU(hidden_size, dropout=0.3, return_sequences=True)
        
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        return out

In [52]:
def make_model(sample, vocab_size=12001, embedding_size=256, hidden_size=1024):
    '''
    모델을 생성함
    
    sample         : 모델의 입력 텐서 차원을 알기 위한 샘플
    vocab_size     : 단어의 가짓 수
    embedding_size : 단어 임베딩의 깊이
    hidden_size    : 모델 내부의 hidden layer의 수
    
    return         : 모델
    '''
    model = TextGenerator(vocab_size, embedding_size , hidden_size)
    model(sample)
    model.summary()
    
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,
                                                              decay_steps=1950,
                                                              decay_rate=0.5)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00015)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    model.compile(loss=loss, optimizer=optimizer)
    return model

# 테스트

In [53]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    '''
    문장이 어떻게 생성되는지 시험, LMS동일
    
    model         : 학습이 완료된 모델
    tokenizer     : 문장을 토큰화 하는데 사용한 맵
    init_sentence : 문장의 시작
    max_len       : 최대 문장의 단어 수
    
    return        : 생성된 문장
    '''
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

# 메인

데이터 준비

In [54]:
num_words = 12000
raw_sentences = read_lyrics()
corpus = make_corpus(raw_sentences)
tensor, tokenizer = tokenize(corpus, num_words)

src_input = tensor[:,:-1]
tgt_input = tensor[:,1:]

train_dataset, val_dataset = make_dataset(src_input, tgt_input, batch_size=32)

원래 문장 TOP 10:
['I saw you walking by his side heard you whisper all those lies',
 "And I couldn't keep from crying",
 'You sang him love songs tenderly that should have been for you and me',
 "And I couldn't keep from crying",
 'I saw his eyes drinking your charms while he held you in his arms',
 'Him with all his wedding ways rules your heart now in my place',
 "I stood and watched him steal a kiss from two lips I know I'll miss",
 "And I couldn't keep from crying I saw his eyes drinking your charms... I "
 "love that hair, long an' black",
 "Hangin' down to the middle of your back",
 "Don't cut it off whatever you do"]


특수 문자, 띄어쓰기 등을 보정한 문장 TOP 10:
['<start> i saw you walking by his side heard you whisper all those lies <end>',
 '<start> and i couldn t keep from crying <end>',
 '<start> you sang him love songs tenderly that should have been for you and '
 'me <end>',
 '<start> and i couldn t keep from crying <end>',
 '<start> i saw his eyes drinking your charms while he held you in

모델 생성

In [55]:
for src_sample, tgt_sample in train_dataset.take(1): break
model = make_model(src_sample, vocab_size=num_words+1, embedding_size=1024, hidden_size=1600)

Model: "text_generator_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  12289024  
_________________________________________________________________
gru_6 (GRU)                  multiple                  12604800  
_________________________________________________________________
gru_7 (GRU)                  multiple                  15369600  
_________________________________________________________________
dense_3 (Dense)              multiple                  19213601  
Total params: 59,477,025
Trainable params: 59,477,025
Non-trainable params: 0
_________________________________________________________________


모델 훈련

In [56]:

model.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
3900/3900 [==============================] - 433s 111ms/step - loss: 3.0882 - val_loss: 2.8007
Epoch 2/10
3900/3900 [==============================] - 432s 111ms/step - loss: 2.6364 - val_loss: 2.5758
Epoch 3/10
3900/3900 [==============================] - 431s 111ms/step - loss: 2.3611 - val_loss: 2.4435
Epoch 4/10
3900/3900 [==============================] - 431s 111ms/step - loss: 2.1433 - val_loss: 2.3513
Epoch 5/10
3900/3900 [==============================] - 432s 111ms/step - loss: 1.9657 - val_loss: 2.2846
Epoch 6/10
3900/3900 [==============================] - 432s 111ms/step - loss: 1.8161 - val_loss: 2.2356
Epoch 7/10
3900/3900 [==============================] - 433s 111ms/step - loss: 1.6887 - val_loss: 2.2013
Epoch 8/10
3900/3900 [==============================] - 433s 111ms/step - loss: 1.5807 - val_loss: 2.1787
Epoch 9/10
3900/3900 [==============================] - 433s 111ms/step - loss: 1.4894 - val_loss: 2.1617
Epoch 10/10
3900/3900 [=======================

생성해보기

In [73]:
i_love = generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)
print(i_love)

you_are = generate_text(model, tokenizer, init_sentence="<start> you are", max_len=20)
print(you_are)

i_am = generate_text(model, tokenizer, init_sentence="<start> i am", max_len=20)
print(i_am)

he_is = generate_text(model, tokenizer, init_sentence="<start> he is", max_len=20)
print(he_is)

when_i = generate_text(model, tokenizer, init_sentence="<start> when i", max_len=20)
print(when_i)

<start> i love you <end> 
<start> you are the only thing that keeps me goin <end> 
<start> i am a god <end> 
<start> he is he is <end> 
<start> when i m with you , all i get is wild thoughts <end> 


----------------------

# 결과들

단어장 12000, batch 256, embedding 256, hidden 1024, LSTM 기본 두개 => loss: 2.2164 - val_loss: 2.5021

단어장 12000, batch 256, embedding 256, hidden 1800, LSTM 기본 두개 => loss: 1.8641 - val_loss: 2.3710, 오버피팅

단어장 20000, batch 256, embedding 256, hidden 1024, LSTM 기본 두개 => loss: 2.3242 - val_loss: 2.6405

단어장 20000, batch 256, embedding 300, hidden 1024, LSTM 기본 두개 => loss: 2.3014 - val_loss: 2.6376

단어장 20000, batch 256, embedding 300, hidden 1200, LSTM 기본 두개 => loss: 2.2094 - val_loss: 2.5762

단어장 12000, batch 256, embedding 256, hidden 1024, LSTM 기본 두개 + Adam lr=0.005 => loss: 2.1292 - val_loss: 2.5862

단어장 12000, batch 256, embedding 256, hidden 1024, GRU 기본 두개 => loss: 1.4231 - val_loss: 2.2650, 오버피팅 심함

단어장 12000, batch 256, embedding 256, hidden 1024, GRU + dropout=0.25 두개 => loss: 1.8833 - val_loss: 2.3803, 오버피팅

단어장 12000, batch 256, embedding 256, hidden 1024, GRU + Adam lr=0.0005 두개 => loss: 2.1238 - val_loss: 2.4549

단어장 12000, batch 256, embedding 512, hidden 1024, GRU + Adam lr=0.0005 두개 => 1.8414 - val_loss: 2.3300

단어장 12000, batch 256, embedding 512, hidden 1024, GRU + dropout=0.5 두개 => loss: 1.7902 - val_loss: 2.3288

단어장 20000, batch 256, embedding 512, hidden 1024, GRU 두개 => 1.9303 - val_loss: 2.4659

단어장 20000, batch 256, embedding 512, hidden 1024, GRU + dropout=0.5 두개 => loss: 2.1578 - val_loss: 2.5078

단어장 20000, batch 128, embedding 512, hidden 1024, GRU 두개 => loss: 1.6931 - val_loss: 2.4475

단어장 12000, batch 128, embedding 512, hidden 1024, GRU + dropout=0.5 두개 => loss: 1.7112 - val_loss: 2.3240

단어장 12000, batch 128, embedding 256, hidden 1024, GRU + dropout=0.5 두개 => loss: 1.7756 - val_loss: 2.3797

단어장 12000, batch 128, embedding 1024, hidden 1024, GRU + dropout=0.5 두개 => 1.6945 - val_loss: 2.3373

단어장 12000, batch 128, embedding 1024, hidden 1024, GRU + dropout=0.5 + lr=0.0008 두개 => loss: 1.6680 - val_loss: 2.2846

단어장 12000, batch 128, embedding 1024, hidden 1024, GRU + dropout=0.5 + lr=0.0006 두개 => loss: 1.6549 - val_loss: 2.2839

단어장 12000, batch 64, embedding 1024, hidden 1024, GRU + dropout=0.5 + lr=0.0006 두개 => loss: 1.6138 - val_loss: 2.2903

단어장 12000, batch 64, embedding 1024, hidden 1024, GRU + dropout=0.5 + lr=0.0004 두개 => loss: 1.6461 - val_loss: 2.2577

단어장 12000, batch 64, embedding 1024, hidden 1024, GRU + dropout=0.4 + lr=0.0004 두개 => loss: 1.5548 - val_loss: 2.2468

단어장 12000, batch 48, embedding 1024, hidden 1024, GRU + dropout=0.5 + lr=0.0004 두개 => loss: 1.6135 - val_loss: 2.2499

단어장 12000, batch 32, embedding 1024, hidden 1024, GRU + dropout=0.5 + lr=0.0003 두개 => loss: 1.6189 - val_loss: 2.2493

단어장 12000, batch 48, embedding 1024, hidden 1024, GRU + dropout=0.5 + lr=0.0001 두개 => loss: 2.2202 - val_loss: 2.4396

단어장 12000, batch 32, embedding 1024, hidden 1024, GRU + dropout=0.5 + lr=0.0001 두개 => loss: 2.1082 - val_loss: 2.3981

단어장 12000, batch 32, embedding 1024, hidden 1024, GRU + dropout=0.4 + lr=0.0001 두개 => loss: 2.0555 - val_loss: 2.3729

단어장 12000, batch 32, embedding 1024, hidden 1024, GRU + dropout=0.3 + lr=0.0001 두개 => loss: 2.0155 - val_loss: 2.3624

단어장 12000, batch 32, embedding 1024, hidden 1600, GRU + dropout=0.3 + lr=0.0001 두개 => loss: 1.6642 - val_loss: 2.2220

### 단어장 12000, batch 32, embedding 1024, hidden 1600, GRU + dropout=0.3 + lr=0.0002 두개 => loss: 1.2835 - val_loss: 2.1723

단어장 12000, batch 64, embedding 512, hidden 1024, GRU + dropout=0.5 + lr=0.0004 두개 => loss: 1.7494 - val_loss: 2.3032

단어장 12000, batch 48, embedding 512, hidden 1024, GRU + dropout=0.5 + lr=0.0004 두개 => loss: 1.6841 - val_loss: 2.2953

단어장 12000, batch 48, embedding 512, hidden 1024, GRU + dropout=0.4 + lr=0.0004 두개 => loss: 1.5877 - val_loss: 2.2765

-----------------------

# 결론

데이터 전처리와 학습 후 새로운 문장 생성 작업에는 LMS의 함수를 그대로 사용하였다.   
길이가 긴 데이터의 처리는 keras의 pad_sequences함수에 옵션으로 부여하는 방법이 아닌 리스트에서 삭제하는 방식을 사용하였다.   
전반적으로 어려움은 없었으나 val_loss를 줄여나가는 부분에서 오랜 시간이 걸렸다.

val_loss를 줄이기 위해서 초기에는 파라미터를 임의로 넣었으나, 낮아지는데 한계를 보였고, 각 파라미터들의 영향을 파악하는데 주력하기로 생각을 바꾸었다.   
단어장 크기는 클수록 늦게 학습되는 경향이 있었고, embedding, hidden과 관련이 있었으므로 12000으로 고정하기로 결정하였다.   
정해진 epoch에서 더 많은 학습을 진행하기 위해서는 batch 크기는 줄여야 했다.   
LSTM은 학습 속도가 늦기 때문에 GRU를 사용하였고, GRU는 학습이 빠른 대신 과적합이 쉽게 일어났기 때문에 dropout을 적용하였다.   
batch크기가 줄어든 만큼 learing rate도 줄여야 과적합을 줄일 수 있었다.   
학습 속도와 과적합 사이에는 비례관계가 있었으므로 10 epoch안에 val_loss 2.2를 달성하기 위해 학습 속도가 적당히 빠르면서도 과적합이 발생하지 않는 파라미터를 찾아야 했다.

최종적으로 "loss: 1.4113 - val_loss: 2.1517"을 얻었고, 이로부터 몇가지 문장을 생성해보았다.   

생성된 문장은 시작 단어에 따라 결과가 다른데, 일반적으로 노래 가사에 쓰이는 단어로 시작시켜야 그럴듯한 문장이 생성되는게 아닌가 생각이 든다.